# DB2-Salesforce connector: Organization information updates

In [1]:
# API settings
api_url = '/services/data/v43.0/sobjects'
object_id = 'organization__c'
external_id = 'Domain__c'

# login parameters to be handled by Papermill
sql_login_params = {"username": "wang2506_ro", "password": "fnVnwcCS7iT45EsA"}
sf_login_params = {
    "grant_type": "password",
    "client_id": "3MVG95jctIhbyCppj0SNJ75IsZ1y8UPGZtSNF4j8FNVXz.De8Lu4jHm3rjRosAtsHy6qjHx3i4S_QbQzvBePG",
    "client_secret": "D1623C6D3607D4FC8004B92C761DFB6C1F70CCD129C5501E357028DFA00F5764",
    "username":"wang2506@purdue.edu",
    "password":"mathisfun992728tXYwCb5yMm6XtohbTRw8ZdBbi",
}



    "username": "wang159-4j1v@force.com",
    "password": "napoleon0eZ3PQpQqE3C3z4wWqEqKGhQ8",

In [2]:
import sys
#sys.path.append(lib_dir)

import pandas as pd
import datetime

## Obtain tool information from DB2 

In [3]:
# connect with DB2
import sqlalchemy as sql

engine = sql.create_engine('mysql+pymysql://%s:%s@127.0.0.1/wang159_myrmekes' \
                                               %(sql_login_params['username'], sql_login_params['password']))

In [7]:
i_char_df = pd.read_sql_query('select * from institution_characteristics', engine)
i_cls_df = pd.read_sql_query('select * from institution_classification', engine)
i_cls_label_df = pd.read_sql_query('select * from institution_classification_labels', engine)
i_cls_var_df = pd.read_sql_query('select * from institution_variable_labels', engine)
usn_ranks_df = pd.read_sql_query('select * from us_news_rankings',engine)

In [8]:
# get domain address from URL
import re

def get_domain(this_domain_name):

    # Replace all non-alphanumeric characters with space
    this_domain_name = re.sub("[^0-9a-zA-Z-]+", ".", this_domain_name.lower())
    name_list = this_domain_name.split('.')

    # index of 'edu'
    try:
        edu_index = name_list.index('edu')
    except:
        return None
    
    if edu_index > 0:
        return name_list[edu_index-1]+'.edu'
    else:
        return None

    

unitid_url_df = i_char_df[['WEBADDR', 'UNITID']].copy()

unitid_url_df.loc[:, 'domain'] = unitid_url_df.WEBADDR.apply(get_domain)

# drop NaN rows
unitid_url_df.dropna(inplace=True)

In [11]:
display(unitid_url_df.head(3))
display(usn_ranks_df.head(2))

,WEBADDR,UNITID,domain
0,www.aamu.edu/,100654,aamu.edu
1,www.uab.edu,100663,uab.edu
2,www.amridgeuniversity.edu,100690,amridgeuniversity.edu


,index,UNITID,INSTNM,WEBADDR,usn_gr_eng,usn_gr_EE,usn_gr_mse,usn_gr_ce,usn_gr_me,usn_gr_ae,...,usn_ug_ME_w_doct,usn_ug_ME_no_doct,usn_ug_ChE_w_doct,usn_ug_ChE_no_doct,usn_ug_IE_w_doct,usn_ug_IE_no_doct,usn_ug_MSE_w_doct,usn_gr_IE,usn_natl,usn_natl_publ
0,0,100654,Alabama A & M University,www.aamu.edu/,NaN,NaN,NaN,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
1,1,100663,University of Alabama at Birmingham,www.uab.edu,NaN,NaN,NaN,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None


In [10]:
# join domain into institution classification table by unitid
df = pd.merge(i_cls_df, unitid_url_df, how='inner', left_on='UNITID', right_on='UNITID')

# display
df.head(3)

,index,UNITID,NAME,CITY,STABBR,CC2000,BASIC2005,BASIC2010,BASIC2015,BASIC2018,...,SATV25,SATM25,SATCMB25,SATACTEQ25,ACTCMP25,ACTFINAL,Unnamed95,Unnamed96,WEBADDR,domain
0,0,177834,A T Still University of Health Sciences,Kirksville,MO,52,25,25,25,25,...,0.0,0.0,0.0,0.0,0.0,0.000000,None,NaN,WWW.ATSU.EDU,atsu.edu
1,1,180203,Aaniiih Nakoda College,Harlem,MT,60,33,33,33,33,...,0.0,0.0,0.0,0.0,0.0,0.000000,None,NaN,www.ancollege.edu,ancollege.edu
2,2,222178,Abilene Christian University,Abilene,TX,21,19,18,18,18,...,510.0,515.0,1025.0,19.0,21.0,20.054054,None,NaN,www.acu.edu,acu.edu


In [12]:
# factoring in information from us news and world report
t_df = pd.merge(df,usn_ranks_df,left_on='UNITID',right_on='UNITID')

display(t_df.head(2))

,index_x,UNITID,NAME,CITY,STABBR,CC2000,BASIC2005,BASIC2010,BASIC2015,BASIC2018,...,usn_ug_ME_w_doct,usn_ug_ME_no_doct,usn_ug_ChE_w_doct,usn_ug_ChE_no_doct,usn_ug_IE_w_doct,usn_ug_IE_no_doct,usn_ug_MSE_w_doct,usn_gr_IE,usn_natl,usn_natl_publ
0,0,177834,A T Still University of Health Sciences,Kirksville,MO,52,25,25,25,25,...,None,None,None,None,None,None,None,None,None,None
1,1,180203,Aaniiih Nakoda College,Harlem,MT,60,33,33,33,33,...,None,None,None,None,None,None,None,None,None,None


In [13]:
def replace_with_label(this_df, c_df):
    this_var = this_df.Variable.iloc[0].strip()
    this_v_l = this_df[['Value', 'Label']]
    
    this_label = pd.merge(c_df[[this_var]], this_v_l, how='left', left_on=this_var, right_on='Value')
    c_df.loc[:, this_var] = this_label['Label']
    
    
i_cls_label_df.groupby('Variable').apply(lambda x: replace_with_label(x, t_df))

""


In [14]:
# display
i_cls_label_df.head(2)

,index,Variable,Value,Label
0,0,CC2000,-3.0,"Not classified, not in classification universe"
1,1,CC2000,15.0,Doctoral/Research Universities—Extensive


In [15]:
# On Salesforce side, domain is unique indentifier of each organization.
# For multiple institutions with same domain (univ. with satellite campuses), use entry with highest total enrollment (FALLENR17)

unique_domain_df = t_df.sort_values(by='FALLENR17', ascending=False).groupby('domain').head(1)

In [16]:
display(unique_domain_df.head(5))
print(unique_domain_df.loc[:,'NAME'])

,index_x,UNITID,NAME,CITY,STABBR,CC2000,BASIC2005,BASIC2010,BASIC2015,BASIC2018,...,usn_ug_ME_w_doct,usn_ug_ME_no_doct,usn_ug_ChE_w_doct,usn_ug_ChE_no_doct,usn_ug_IE_w_doct,usn_ug_IE_no_doct,usn_ug_MSE_w_doct,usn_gr_IE,usn_natl,usn_natl_publ
3687,3849,484613,University of Phoenix-Arizona,Tempe,AZ,"Not classified, not in classification universe","Not classified, not in classification universe",NaN,Doctoral Universities: Moderate Research Activity,Doctoral/Professional Universities,...,None,None,None,None,None,None,None,None,None,None
3996,4171,433387,Western Governors University,Salt Lake City,UT,"Not classified, not in classification universe",NaN,Master's L: Master's Colleges and Universities...,Master's Colleges & Universities: Larger Programs,Master's Colleges & Universities: Larger Programs,...,None,None,None,None,None,None,None,None,None,None
3095,3235,183026,Southern New Hampshire University,Manchester,NH,Specialized Institutions—Schools of business a...,NaN,Master's L: Master's Colleges and Universities...,Master's Colleges & Universities: Larger Programs,Master's Colleges & Universities: Larger Programs,...,None,None,None,None,None,None,None,None,None,None
1399,1458,104717,Grand Canyon University,Phoenix,AZ,Master's Colleges and Universities I,NaN,Master's L: Master's Colleges and Universities...,Doctoral Universities: Moderate Research Activity,Doctoral/Professional Universities,...,None,None,None,None,None,None,None,None,None,None
1653,1723,150987,Ivy Tech Community College,Indianapolis,IN,Associate's Colleges,NaN,Assoc/Pub-U-SC: Associate's--Public Urban-serv...,Associate's Colleges: Mixed Transfer/Career & ...,Associate's Colleges: Mixed Transfer/Career & ...,...,None,None,None,None,None,None,None,None,None,None


3687                        University of Phoenix-Arizona
3996                         Western Governors University
3095                    Southern New Hampshire University
1399                              Grand Canyon University
1653                           Ivy Tech Community College
                              ...                        
501                          California Christian College
2160    Monteclaro Escuela de Hoteleria y Artes Culina...
1881                Los Angeles Academy of Figurative Art
3411    Theological Seminary of the Reformed Episcopal...
1451                         Harrington College of Design
Name: NAME, Length: 3342, dtype: object


## Obtain organization metadata from Salesforce

In [2]:
from DB2SalesforceAPI import DB2SalesforceAPI

# create DB2 to Salesforce API object
db_s = DB2SalesforceAPI(sf_login_params)

# specify Salesforce object ID and external ID
org_metadata = db_s.get_obj_metadata('Organization__c')

Obtained Salesforce access token ...... True


In [20]:
import json

# get all fields
field_list = json.loads(org_metadata)['fields']

# list of available field API names
available_fields = list()

for this_field in field_list:
    this_name = this_field['name']
    
    if len(this_name) > 3:
        if this_name[-3:] == '__c':
            # this is a custom-made field
            available_fields.append(this_name[:-3])
            
# display
display(available_fields)

['Domain',
 'Country',
 'Industry',
 'Size_Range',
 'MSI',
 'is_US_institution',
 'HBCU',
 'BASIC2018',
 'usn_gr_eng',
 'usn_gr_EE',
 'usn_gr_mse',
 'usn_gr_ce',
 'usn_gr_me',
 'usn_gr_ae',
 'usn_gr_IE',
 'chem_gr_anal',
 'chem_gr_inorg',
 'chem_gr_phy',
 'chem_gr_bio',
 'chem_gr_org',
 'chem_gr_theo',
 'phys_gr_atom',
 'phys_gr_con',
 'phys_gr_quan',
 'usn_ug_eng_w_doct',
 'usn_ug_eng_no_doctorate',
 'usn_ug_EE_w_doct',
 'usn_ug_EE_no_doct',
 'usn_ug_CE_w_doct',
 'usn_ug_CE_no_doct',
 'usn_ug_ME_w_doct',
 'usn_ug_ME_no_doct',
 'usn_ug_ChE_w_doct',
 'usn_ug_ChE_no_doct',
 'usn_ug_IE_w_doct',
 'usn_ug_IE_no_doct',
 'usn_ug_MSE_w_doct',
 'usn_natl',
 'usn_natl_publ',
 'usn_ug_MSE_no_doct']

## Match data with Salesforce format

In [23]:
# find columns that have matched fields in Salesforce
matched_columns = set(unique_domain_df.columns).intersection(set(available_fields))

# display
display(matched_columns)

{'BASIC2018',
 'HBCU',
 'MSI',
 'chem_gr_anal',
 'chem_gr_bio',
 'chem_gr_inorg',
 'chem_gr_org',
 'chem_gr_phy',
 'chem_gr_theo',
 'phys_gr_atom',
 'phys_gr_con',
 'phys_gr_quan',
 'usn_gr_EE',
 'usn_gr_IE',
 'usn_gr_ae',
 'usn_gr_ce',
 'usn_gr_eng',
 'usn_gr_me',
 'usn_gr_mse',
 'usn_natl',
 'usn_natl_publ',
 'usn_ug_CE_no_doct',
 'usn_ug_CE_w_doct',
 'usn_ug_ChE_no_doct',
 'usn_ug_ChE_w_doct',
 'usn_ug_EE_no_doct',
 'usn_ug_EE_w_doct',
 'usn_ug_IE_no_doct',
 'usn_ug_IE_w_doct',
 'usn_ug_ME_no_doct',
 'usn_ug_ME_w_doct',
 'usn_ug_MSE_w_doct',
 'usn_ug_eng_no_doctorate',
 'usn_ug_eng_w_doct'}

In [24]:
df_sf = pd.DataFrame()

# Make sure NaN and NaT values are taken care of here

df_sf['Domain__c'] = unique_domain_df['domain']

df_sf['is_US_institution__c'] = True
df_sf['Name'] = unique_domain_df['NAME']

# Transfer all matched columns to df_sf
for this_col in matched_columns:
    df_sf[this_col+'__c'] = unique_domain_df[this_col]

sf_original_fields = df_sf.columns

# display
df_sf.head(2).T

,3687,3996
Domain__c,phoenix.edu,wgu.edu
is_US_institution__c,True,True
Name,University of Phoenix-Arizona,Western Governors University
usn_ug_EE_w_doct__c,None,None
chem_gr_anal__c,NaN,NaN
usn_natl_publ__c,None,None
usn_ug_EE_no_doct__c,None,None
usn_ug_ME_w_doct__c,None,None
usn_ug_ME_no_doct__c,None,None
HBCU__c,No,No


## To Salesforce Sales Cloud CRM

In [39]:
from DB2SalesforceAPI import DB2SalesforceAPI

# create DB2 to Salesforce API object
db_s = DB2SalesforceAPI(sf_login_params)

# specify Salesforce object ID and external ID
db_s.object_id = object_id
db_s.external_id = external_id

Obtained Salesforce access token ...... True


In [40]:
display(df_sf.head(2))

,Domain__c,is_US_institution__c,Name,phys_gr_atom__c,chem_gr_anal__c,phys_gr_con__c,usn_ug_ME_no_doct__c,usn_gr_IE__c,usn_gr_ae__c,chem_gr_bio__c,...,chem_gr_theo__c,usn_ug_IE_w_doct__c,usn_ug_ME_w_doct__c,usn_natl__c,phys_gr_quan__c,usn_gr_EE__c,usn_ug_MSE_w_doct__c,usn_ug_ChE_no_doct__c,MSI__c,usn_ug_EE_w_doct__c
3687,phoenix.edu,True,University of Phoenix-Arizona,NaN,NaN,NaN,None,None,NaN,NaN,...,NaN,None,None,None,NaN,NaN,None,None,No,None
3996,wgu.edu,True,Western Governors University,NaN,NaN,NaN,None,None,NaN,NaN,...,NaN,None,None,None,NaN,NaN,None,None,No,None


In [41]:
# send data to Salesforce
db_s.send_data(df_sf)

[Success] Bulk job creation successful. Job ID = 7505w00000Mv5T6AAJ
hello
[Success] CSV upload successful. Job ID = 7505w00000Mv5T6AAJ
[Success] Closing job successful. Job ID = 7505w00000Mv5T6AAJ


In [42]:
# check status
db_s.check_bulk_status()

{'id': '7505w00000Mv5T6AAJ',
 'operation': 'upsert',
 'object': 'organization__c',
 'createdById': '0055w00000ArpYvAAJ',
 'createdDate': '2020-06-16T15:23:11.000+0000',
 'systemModstamp': '2020-06-16T15:23:12.000+0000',
 'state': 'UploadComplete',
 'externalIdFieldName': 'Domain__c',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 0,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apexProcessingTime': 0}

In [43]:
from pprint import pprint
pprint(db_s.check_bulk_failed_results())

''
